Copyright **`(c)`** 2023 Alessandro De Marco`<a.demarco@studenti.polito.it>`  
[Github](https://github.com/Aleedm)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/Aleedm/computational-intelligence/blob/3e7d43ccabdc267b53f693df02c6b39eecc2f21f/LICENSE) for details.  


# LAB10

Use reinforcement learning to devise a tic-tac-toe player.

### Deadlines:

* Submission: [Dies Natalis Solis Invicti](https://en.wikipedia.org/wiki/Sol_Invictus)
* Reviews: [Befana](https://en.wikipedia.org/wiki/Befana)

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [ ]:
from itertools import combinations
from collections import namedtuple, defaultdict
from random import choice, random, choices
from copy import deepcopy
import copy

from tqdm.auto import tqdm
import numpy as np

State = namedtuple('State', ['x', 'o', 'turn'])
MAGIC = [2, 7, 6, 
         9, 5, 1, 
         4, 3, 8]

# Hyperparameters

In [ ]:
episodes = 10_000_000
selection_factor = 3

# Usefull functions

In [ ]:
def print_board(pos):
    """Nicely prints the board"""
    for r in range(3):
        for c in range(3):
            i = r * 3 + c
            if MAGIC[i] in pos.x:
                print('X', end='')
            elif MAGIC[i] in pos.o:
                print('O', end='')
            else:
                print('.', end='')
        print()
    print()
    
def win(elements):
    """Checks is elements is winning"""
    return any(sum(c) == 15 for c in combinations(elements, 3))

def state_value(pos: State):
    """Evaluate state: +1 first player wins"""
    if win(pos.x):
        return 1
    elif win(pos.o):
        return -1
    else:
        return 0

In [ ]:
def normalize_weights(weights):
    total = sum(weights)
    if total == 0:
        return [1/len(weights) for _ in weights]  
    return [w / total for w in weights]

# Q-Learning Implementation

In [ ]:
def create_state_action_key(game_state, action, next_turn=None):
    if next_turn is None:
        next_turn = game_state.turn
    if next_turn == 'o':
        x_positions = frozenset(game_state.x).union([action])
        o_positions = frozenset(game_state.o)
    else:
        x_positions = frozenset(game_state.x)
        o_positions = frozenset(game_state.o).union([action])
    return (x_positions, o_positions, action, next_turn)

In [ ]:

def select_best_action_q_learning(game_state, q_values, select_best=True):
    available_actions = [a for a in MAGIC if a not in game_state.x and a not in game_state.o]
    if not available_actions:
        return None
    action_values = []
    next_turn = 'o' if game_state.turn == 'x' else 'x'

    for action in available_actions:
        key = create_state_action_key(game_state, action, next_turn)
        value = q_values[key]
        action_values.append((action, value))

    action_values.sort(key=lambda x: x[1], reverse=(game_state.turn == 'x'))
    
    n_best_actions = len(available_actions) // selection_factor + (len(available_actions) % selection_factor > 0)
    best_actions = action_values[:n_best_actions]
    
    if select_best:
        return best_actions[0][0]

    if game_state.turn == 'x':
        weights = [value for _, value in best_actions]
    else:
        max_value = max(value for _, value in best_actions)
        weights = [max_value - value + 0.1 for _, value in best_actions]
        
    normalized_weights = normalize_weights(weights)
    
    selected_action = choices([action for action, _ in best_actions], weights=normalized_weights, k=1)[0]

    return selected_action

In [ ]:
def generate_random_game(q_values, epsilon=1):
    episode = []
    player_x_positions = set()
    player_o_positions = set()
    current_turn = 'x'
    available_actions = set(MAGIC)

    while available_actions:
        if random() < epsilon:
            action = choice(list(available_actions))
        else:
            game_state = State(player_x_positions, player_o_positions, current_turn)
            action = select_best_action_q_learning(game_state, q_values)

        if current_turn == 'x':
            player_x_positions.add(action)
            current_turn = 'o'
        else:
            player_o_positions.add(action)
            current_turn = 'x'
        episode_state = State(copy.copy(player_x_positions), copy.copy(player_o_positions), current_turn)
        episode.append((episode_state, action))
        available_actions.remove(action)

        if win(player_x_positions) or win(player_o_positions) or not available_actions:
            break

    return episode


In [ ]:
learning_rate = 0.1
discount_factor = 0.9

def train_q_learning(total_episodes, learning_rate, discount_factor):
    q_values = defaultdict(float)
    
    epsilon = 1
    epsilon_min = 0.3
    epsilon_decay = (epsilon - epsilon_min) / (total_episodes // 2)
    
    for episode_number in tqdm(range(total_episodes)):
        if episode_number >= total_episodes // 2:
            epsilon = max(epsilon - epsilon_decay, epsilon_min)
        episode = generate_random_game(q_values, epsilon)
        final_reward = state_value(episode[-1][0])
        for game_state, action in reversed(episode):
            key = create_state_action_key(game_state, action)
            future_rewards = [q_values[create_state_action_key(game_state, a)] for a in MAGIC if a not in game_state.x and a not in game_state.o]
            max_future_reward = max(future_rewards, default=0)
            q_values[key] = q_values[key] + learning_rate * (final_reward + discount_factor * max_future_reward - q_values[key])
            final_reward = q_values[key] 
    return q_values


In [ ]:
q_values = train_q_learning(episodes, learning_rate, discount_factor)

# Monte Carlo Implementation

In [ ]:
def select_best_action_monte_carlo(game_state, state_values):
    available_actions = [a for a in MAGIC if a not in game_state.x and a not in game_state.o]
    if not available_actions:
        return None
    action_value_pairs = []

    for action in available_actions:
        next_state = State(
            frozenset(game_state.x.union([action])) if game_state.turn == 'x' else game_state.x, 
            frozenset(game_state.o.union([action])) if game_state.turn == 'o' else game_state.o, 
            'o' if game_state.turn == 'x' else 'x'
        )
        value = state_values.get(next_state, 0)
        action_value_pairs.append((action, value))

    action_value_pairs.sort(key=lambda x: x[1], reverse=(game_state.turn == 'x'))
    selection_factor = 3
    n_best_actions = len(available_actions) // selection_factor + (len(available_actions) % selection_factor > 0)
    best_actions = action_value_pairs[:n_best_actions]

    if game_state.turn == 'x':
        weights = [value for _, value in best_actions]
    else:
        max_value = max(value for _, value in best_actions)
        weights = [max_value - value + 0.1 for _, value in best_actions]

    normalized_weights = normalize_weights(weights)
    selected_action = choices([action for action, _ in best_actions], weights=normalized_weights, k=1)[0]

    return selected_action

In [ ]:
def generate_monte_carlo_episode():
    episode = []
    game_state = State(frozenset(), frozenset(), 'x')
    available_actions = set(MAGIC)

    while True:
        action = choice(list(available_actions))
        available_actions.remove(action)
        
        if game_state.turn == 'x':
            new_x_positions = game_state.x.union([action])
            game_state = State(frozenset(new_x_positions), game_state.o, 'o')
        else:
            new_o_positions = game_state.o.union([action])
            game_state = State(game_state.x, frozenset(new_o_positions), 'x')

        episode.append(game_state)

        if win(game_state.x) or win(game_state.o) or len(game_state.x) + len(game_state.o) == 9:
            break

    reward = 1 if win(game_state.x) else -1 if win(game_state.o) else 0
    return [(s, reward) for s in episode]

In [ ]:
def train_monte_carlo(total_episodes):
    state_values = defaultdict(float)
    state_counts = defaultdict(int)

    for _ in tqdm(range(total_episodes)):
        episode = generate_monte_carlo_episode()
        total_return = 0
        for game_state, reward in reversed(episode):
            total_return = reward + total_return
            state_counts[game_state] += 1
            state_values[game_state] += (total_return - state_values[game_state]) / state_counts[game_state]

    return state_values


In [ ]:
state_values_mc = train_monte_carlo(episodes)

# Results

In [ ]:
def random_agent(state, _):
    available_moves = [a for a in MAGIC if a not in state.x and a not in state.o]
    return choice(available_moves)


In [ ]:
def update_state(state, move):
    new_x = state.x.union([move]) if state.turn == 'x' else state.x
    new_o = state.o.union([move]) if state.turn == 'o' else state.o
    new_turn = 'o' if state.turn == 'x' else 'x'
    return State(new_x, new_o, new_turn)

def is_terminal(state):
    return win(state.x) or win(state.o) or len(state.x) + len(state.o) == 9

In [ ]:
def update_results(state, results, last_player, agent1_starts, agent1_name, agent2_name, initial_moves, turn_count):
    if win(state.x):
        winner = agent1_name if agent1_starts else agent2_name
        loser = agent2_name if agent1_starts else agent1_name
        result = "Win"
    elif win(state.o):
        winner = agent2_name if agent1_starts else agent1_name
        loser = agent1_name if agent1_starts else agent2_name
        result = "Loss"
    else:
        winner = loser = None
        result = "Draw"

    game_result_info_w = {
        "Initial Moves": initial_moves,
        "Turn Count": turn_count,
        "Result": "Win"
    }
    game_result_info_l = {
        "Initial Moves": initial_moves,
        "Turn Count": turn_count,
        "Result": "Loss"
    }

    if winner:
        if agent1_name == winner and agent1_starts:
            results[agent1_name]["Starts First"]["Games"].append(game_result_info_w)
            results[agent2_name]["Starts Second"]["Games"].append(game_result_info_l)
        elif agent1_name == winner and not agent1_starts:
            results[agent1_name]["Starts Second"]["Games"].append(game_result_info_w)
            results[agent2_name]["Starts First"]["Games"].append(game_result_info_l)
        elif agent2_name == winner and agent1_starts:
            results[agent2_name]["Starts Second"]["Games"].append(game_result_info_w)
            results[agent1_name]["Starts First"]["Games"].append(game_result_info_l)
        elif agent2_name == winner and not agent1_starts:
            results[agent2_name]["Starts First"]["Games"].append(game_result_info_w)
            results[agent1_name]["Starts Second"]["Games"].append(game_result_info_l)
    else:
        game_result_info = {
            "Initial Moves": initial_moves,
            "Turn Count": turn_count,
            "Result": "Draw"
        }
        results[agent1_name]["Starts First" if agent1_starts else "Starts Second"]["Games"].append(game_result_info)
        results[agent2_name]["Starts Second" if agent1_starts else "Starts First"]["Games"].append(game_result_info)


def compete(num_games, agent1_name, choose_move_agent1, agent1_dict, agent2_name, choose_move_agent2, agent2_dict):
    results = {
        agent1_name: {
            "Starts First": {"Games": []},
            "Starts Second": {"Games": []}
        },
        agent2_name: {
            "Starts First": {"Games": []},
            "Starts Second": {"Games": []}
        }
    }
    agent1_starts = True 

    for _ in range(num_games):
        state = State(frozenset(), frozenset(), 'x')
        is_agent1_turn = agent1_starts
        turn_count = 0
        initial_moves = []

        while True:
            if is_agent1_turn:
                move = choose_move_agent1(state, agent1_dict)
                player = agent1_name
            else:
                move = choose_move_agent2(state, agent2_dict)
                player = agent2_name

            if turn_count < 2:
                initial_moves.append(move)

            state = update_state(state, move)
            turn_count += 1

            if is_terminal(state):
                update_results(state, results, player, agent1_starts, agent1_name, agent2_name, initial_moves, turn_count)
                break

            is_agent1_turn = not is_agent1_turn
        agent1_starts = not agent1_starts

    return results

In [ ]:
num_games = 100_000
results_mc_ql = compete(num_games, "Montecarlo", select_best_action_monte_carlo, state_values_mc, "Q-Learning", select_best_action_q_learning, q_values)
results_mc_rn = compete(num_games, "Montecarlo", select_best_action_monte_carlo, state_values_mc, "Random", random_agent, None)
results_ql_rn = compete(num_games, "Q-Learning", select_best_action_q_learning, q_values, "Random", random_agent, None)

In [ ]:
#Plot montecarlo vs q-learning
plot_total_outcomes(results_mc_ql, "total_outcomes_mc_ql.png")
plot_outcomes_by_starting_agent(results_mc_ql, "outcomes_by_starting_agent_mc_ql.png")
plot_wins_draws_by_turns(results_mc_ql, "wins_draws_by_turns_mc_ql.png")
plot_top_two_moves_podium(results_mc_ql, "top_two_moves_podium_mc_ql.png")
#Plot montecarlo vs random
plot_total_outcomes(results_mc_rn, "total_outcomes_mc_rn.png")
plot_outcomes_by_starting_agent(results_mc_rn, "outcomes_by_starting_agent_mc_rn.png")
plot_wins_draws_by_turns(results_mc_rn, "wins_draws_by_turns_mc_rn.png")
plot_top_two_moves_podium(results_mc_rn, "top_two_moves_podium_mc_rn.png")
#Plot q-learning vs random
plot_total_outcomes(results_ql_rn, "total_outcomes_ql_rn.png")
plot_outcomes_by_starting_agent(results_ql_rn, "outcomes_by_starting_agent_ql_rn.png")
plot_wins_draws_by_turns(results_ql_rn, "wins_draws_by_turns_ql_rn.png")
plot_top_two_moves_podium(results_ql_rn, "top_two_moves_podium_ql_rn.png")

# Plot functions

In [ ]:

import matplotlib.pyplot as plt

def plot_total_outcomes(results, filename="total_outcomes.png"):
    agents_name = list(results.keys())
    total_wins = {agent: 0 for agent in results}
    total_draws = 0

    for agent, data in results.items():
        for position in ["Starts First", "Starts Second"]:
            for game in data[position]["Games"]:
                if game["Result"] == "Win":
                    total_wins[agent] += 1
                elif game["Result"] == "Draw":
                    total_draws += 1

    fig, ax = plt.subplots()
    outcomes = [f'{agents_name[0]} Wins', f'{agents_name[1]} Wins', 'Draws']
    values = [total_wins[agents_name[0]], total_wins[agents_name[1]], total_draws/2]

    ax.bar(outcomes, values)

    ax.set_ylabel('Number of Games')
    ax.set_title('Total Game Outcomes')
    plt.savefig(filename)
    plt.show()

In [ ]:
def plot_outcomes_by_starting_agent(results, filename="outcomes_by_starting_agent.png"):
    agents_name = list(results.keys())
    outcomes_first = {"Win": 0, "Loss": 0, "Draw": 0}
    outcomes_second = {"Win": 0, "Loss": 0, "Draw": 0}

    for game in results[agents_name[0]]["Starts First"]["Games"]:
        outcomes_first[game["Result"]] += 1

    for game in results[agents_name[1]]["Starts First"]["Games"]:
        outcomes_second[game["Result"]] += 1

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

    ax1.bar(outcomes_first.keys(), outcomes_first.values())
    ax1.set_title(f"{agents_name[0]} Starts First")
    ax1.set_ylabel("Number of Games")
    
    ax2.bar(outcomes_second.keys(), outcomes_second.values())
    ax2.set_title(f"{agents_name[1]} Starts First")

    plt.savefig(filename)
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_wins_draws_by_turns(results, filename="wins_draws_by_turns.png"):
    agents_name = list(results.keys())
    turn_counts = list(range(5, 10))
    outcomes = ["Win", "Draw", "Loss"]
    data = {agent: {outcome: [0 for _ in turn_counts] for outcome in outcomes} for agent in results}

    for agent, positions in results.items():
        for position, games in positions.items():
            for game in games["Games"]:
                if game["Turn Count"] in turn_counts:
                    data[agent][game["Result"]][game["Turn Count"] - 5] += 1

    n_groups = len(turn_counts)
    fig, ax = plt.subplots()

    index = np.arange(n_groups)
    bar_width = 0.35
    opacity = 0.8

    rects1 = ax.bar(index, data[agents_name[0]]["Win"], bar_width,
                    alpha=opacity, label=f'{agents_name[0]} Wins')

    rects2 = ax.bar(index, data[agents_name[0]]["Draw"], bar_width, bottom=data[agents_name[0]]["Win"],
                    alpha=opacity, label='Draws')

    rects3 = ax.bar(index + bar_width, data[agents_name[1]]["Win"], bar_width,
                    alpha=opacity, label=f'{agents_name[1]} Wins')

    ax.set_xlabel('Turn Count')
    ax.set_ylabel('Scores')
    ax.set_title('Scores by turn count and outcome')
    ax.set_xticks(index + bar_width / 2)
    ax.set_xticklabels(turn_counts)
    ax.legend()

    plt.tight_layout()
    plt.savefig(filename)
    plt.show()



In [ ]:
def plot_top_two_moves_podium(results, filename_prefix="top_two_moves_podium"):
    top_moves = {agent: Counter() for agent in results}
    top_moves_draws = Counter()

    for agent, data in results.items():
        for position in ["Starts First", "Starts Second"]:
            for game in data[position]["Games"]:
                if len(game["Initial Moves"]) >= 2:
                    move_combination = tuple(game["Initial Moves"][:2])
                    if game["Result"] == "Win":
                        top_moves[agent][move_combination] += 1
                    elif game["Result"] == "Draw":
                        top_moves_draws[move_combination] += 1

    podium_data = {agent: top_moves[agent].most_common(3) for agent in results}
    podium_data["Draws"] = top_moves_draws.most_common(3)

    def draw_board(ax, moves, title):
        ax.plot([1, 1], [0, 3], 'k', linewidth=2)
        ax.plot([2, 2], [0, 3], 'k', linewidth=2)
        ax.plot([0, 3], [1, 1], 'k', linewidth=2)
        ax.plot([0, 3], [2, 2], 'k', linewidth=2)
        for i, move in enumerate(moves):
            symbol = 'X' if i % 2 == 0 else 'O'
            row, col = divmod(move - 1, 3)
            ax.text(col + 0.5, 2 - row + 0.5, symbol, fontsize=36, ha='center', va='center')
        ax.set_title(title)
        ax.axis('off')

    for agent, top_moves in podium_data.items():
        fig, axs = plt.subplots(1, 3, figsize=(15, 5))
        fig.suptitle(f'Top Two Moves for {agent}', fontsize=16)

        for i, (moves, count) in enumerate(top_moves):
            draw_board(axs[i], moves, f'Rank {i+1}: {count} wins')

        plt.tight_layout()
        plt.savefig(f"{filename_prefix}_{agent}.png")
        plt.show()

# Play with the agent

In [ ]:
import os
from IPython.display import clear_output
import time

def clear_console():
    os.system('cls' if os.name == 'nt' else 'clear')
    clear_output(wait=True)
    time.sleep(0.1)


In [ ]:
def play_against_agent(choose_move_agent, agent_dict):
    state = State(frozenset(), frozenset(), 'x')

    while not is_terminal(state):
        if state.turn == 'x':
            print_board(state)
            print(f"\n{MAGIC[0:3]}\n{MAGIC[3:6]}\n{MAGIC[6:9]}")
            move = int(input(f"Scegli la tua mossa (1-9):"))
            while move not in MAGIC or move in state.x or move in state.o:
                print("Mossa non valida. Riprova.")
                move = int(input("Scegli la tua mossa (1-9): "))
            state = State(state.x.union([move]), state.o, 'o')
        else:
            move = choose_move_agent(state, agent_dict)
            state = State(state.x, state.o.union([move]), 'x')
        clear_console()

    print_board(state)
    if win(state.x):
        print("Hai vinto!")
    elif win(state.o):
        print("L'agente ha vinto.")
    else:
        print("Pareggio.")


def print_board_with_choices(state):
    for r in range(3):
        for c in range(3):
            i = r * 3 + c + 1
            if i in state.x:
                print('X', end=' ')
            elif i in state.o:
                print('O', end=' ')
            else:
                print(MAGIC[i], end=' ')
        print()

In [ ]:
play_against_agent(select_best_action_monte_carlo, state_values_mc)

In [ ]:
def compete_interactive(num_games, q_function, state_values):
    monte_carlo_starts = True  

    for game in range(num_games):
        print(f"Partita {game + 1}")
        state = State(frozenset(), frozenset(), 'x')
        turn = monte_carlo_starts
        while True:
            if turn:
                player = "Monte Carlo"
                move = select_best_action_monte_carlo(state, state_values)
            else:
                player = "Q-learning"
                move = select_best_action_q_learning(state, q_function)

            state = update_state(state, move)
            print(f"Mossa di {player}: {move}")
            print_board(state)

            if is_terminal(state):
                winner = state_value(state)
                print(f"Vince {player}!" if winner == 1 else "Pareggio!" if winner == 0 else f"Vince {player}!")
                break

            input("Premi INVIO per continuare...") 
            turn = not turn
        monte_carlo_starts = not monte_carlo_starts


In [ ]:
compete_interactive(2, q_values, state_values_mc)